# Practical Exam: House Sales

## Introduction

RealAgents, a real estate company, aims to optimize house listing prices to minimize time to sale. We'll use a dataset of previous house sales to predict sale prices based on house characteristics.

## Data

The dataset (house_sales.csv) includes the following columns:

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. Unique identifier for houses. Missing values not possible. |
| city        | Nominal. The city in which the house is located (one of 'Silvertown', 'Riverford', 'Teasdale', 'Poppleton'). Replace missing values with "Unknown". |
| sale_price  | Discrete. The sale price of the house in whole dollars. Remove missing entries. |
| sale_date   | Discrete. The date of the last sale of the house. Replace missing values with 2023-01-01. |
| months_listed  | Continuous. The number of months the house was listed prior to its last sale, rounded to one decimal place. Replace missing values with the mean. |
| bedrooms    | Discrete. The number of bedrooms. Replace missing values with the mean, rounded to the nearest integer. |
| house_type   | Ordinal. One of "Terraced", "Semi-detached", or "Detached". Replace missing values with the most common type. |
| area      | Continuous. The area in square meters, rounded to one decimal place. Replace missing values with the mean. |

## Tasks 

We will complete the following tasks:

1. **Identify Missing 'city' Values:** Calculate the number of missing values in the `city` column.
2. **Data Cleaning and Preprocessing:** Prepare the data for modeling.
3. **Average Sale Price by Bedrooms:**  Calculate the average and variance of sale price for each number of bedrooms. 
4. **Baseline Model:** Fit a linear regression model to predict sale price. 
5. **Comparison Model:** Fit a decision tree model for comparison.

In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

## Task 1: Calculate Missing 'city' Values 

##### - Calculates and prints the number of missing values in the city column of the house_sales.csv dataset.
##### - Identifies potential data quality issues. Missing values could indicate incomplete data collection or errors in recording. This helps understand the completeness of the dataset.

In [6]:
df = pd.read_csv("Downloads\\house_sales.csv")
missing_city = (df['city'] == "--").sum() 
print(f"Task 1: Missing 'city' values: {missing_city}")

train_data, validation_data = train_test_split(df, test_size=0.2, random_state=42) 
train_data.to_csv("train.csv", index=False)
validation_data.to_csv("validation.csv", index=False)

Task 1: Missing 'city' values: 73


## Task 2: Data Cleaning and Preprocessing

day).

In [31]:
def clean_data(df):
    """Cleans and preprocesses the input DataFrame."""

    df = df.copy()

    # Replace missing values in 'city'
    df['city'] = df['city'].fillna('Unknown')

    # Remove rows with missing 'sale_price'
    df = df.dropna(subset=['sale_price'])

    # Replace missing 'sale_date'
    df['sale_date'] = df['sale_date'].fillna('2023-01-01')

    # Replace missing 'months_listed' with the mean
    df['months_listed'] = df['months_listed'].fillna(round(df['months_listed'].mean(), 1))

    # Replace missing 'bedrooms' with the mean
    df['bedrooms'] = df['bedrooms'].fillna(round(df['bedrooms'].mean()))

    # Replace abbreviations in 'house_type'
    df['house_type'] = df['house_type'].replace({'Det.': 'Detached', 'Semi': 'Semi-detached', 'Terr.': 'Terraced'})

    # Replace missing 'house_type' with the mode
    df['house_type'] = df['house_type'].fillna(df['house_type'].mode()[0])

    # Remove unit from 'area' and convert to float
    df['area'] = df['area'].apply(lambda x: x.replace(' sq.m.', '') if isinstance(x, str) else x).astype(float)

    # Convert 'sale_date' to datetime
    df['sale_date'] = pd.to_datetime(df['sale_date'])

    # Extract numerical features from 'sale_date'
    df['sale_year'] = df['sale_date'].dt.year
    df['sale_month'] = df['sale_date'].dt.month
    df['sale_day'] = df['sale_date'].dt.day

    # Drop the original 'sale_date' column
    df = df.drop('sale_date', axis=1) 

    return df

df_cleaned = clean_data(df)
train_data_cleaned = clean_data(train_data)  
validation_data_cleaned = clean_data(validation_data)
print(df_cleaned.head())
print("Task 2: Data cleaned.")

   house_id        city  sale_price  months_listed  bedrooms     house_type  \
0   1217792  Silvertown       55943            5.4         2  Semi-detached   
1   1900913  Silvertown      384677            6.3         5       Detached   
2   1174927   Riverford      281707            6.9         6       Detached   
3   1773666  Silvertown      373251            6.1         6       Detached   
4   1258487  Silvertown      328885            8.7         5       Detached   

    area  sale_year  sale_month  sale_day  
0  107.8       2021           9        12  
1  498.8       2021           1        17  
2  542.5       2021          11        10  
3  528.4       2020           4        13  
4  477.1       2020           9        24  
Task 2: Data cleaned.


## Task 3: Average Sale Price by Bedrooms

In [11]:
price_by_rooms = df_cleaned.groupby('bedrooms')['sale_price'].agg(['mean', 'var']).reset_index()
price_by_rooms.columns = ['bedrooms', 'avg_price', 'var_price']
price_by_rooms = price_by_rooms.round(1)
print("Task 3: Price by rooms calculated:")
print(price_by_rooms)

Task 3: Price by rooms calculated:
   bedrooms  avg_price     var_price
0         2    67076.4  5.652896e+08
1         3   154665.1  2.378289e+09
2         4   234704.6  1.725211e+09
3         5   301515.9  2.484328e+09
4         6   375741.3  3.924432e+09


## Task 4: Model Fitting and Prediction

In [13]:
def model_1():

    # Clean data
    train_data_cleaned = clean_data(train_data)
    validation_data_cleaned = clean_data(validation_data)

    # Feature and Target Selection
    features = ['bedrooms', 'area', 'months_listed'] 
    target = 'sale_price'

    X_train = train_data_cleaned[features]
    y_train = train_data_cleaned[target]
    X_val = validation_data_cleaned[features]

    # Model training using Linear Regression
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Prediction
    predictions = model.predict(X_val)

    # Create Result DataFrame
    base_result = pd.DataFrame({'house_id': validation_data_cleaned['house_id'], 'price': predictions})
    return base_result

## Task 5: Comparison Model (Decision Tree Regressor)

In [15]:
def model_2():
    # Clean data
    train_data_cleaned = clean_data(train_data)
    validation_data_cleaned = clean_data(validation_data)

    # One-hot Encode Categorical Features
    categorical_cols = ['city', 'house_type']
    enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    train_encoded = pd.DataFrame(enc.fit_transform(train_data_cleaned[categorical_cols]))
    val_encoded = pd.DataFrame(enc.transform(validation_data_cleaned[categorical_cols]))
    
    train_encoded.columns = enc.get_feature_names_out(categorical_cols)
    val_encoded.columns = enc.get_feature_names_out(categorical_cols)
    
    train_data_cleaned = train_data_cleaned.drop(categorical_cols, axis=1).reset_index(drop=True).join(train_encoded)
    validation_data_cleaned = validation_data_cleaned.drop(categorical_cols, axis=1).reset_index(drop=True).join(val_encoded)

    # Feature and Target Selection (All features)
    features = train_data_cleaned.columns.drop('sale_price') 
    target = 'sale_price'
    X_train = train_data_cleaned[features]
    y_train = train_data_cleaned[target]
    X_val = validation_data_cleaned[features]

    # Model Training (Decision Tree)
    model = DecisionTreeRegressor(random_state=42)  
    model.fit(X_train, y_train)

    # Prediction
    predictions = model.predict(X_val)

    # Create Result DataFrame
    compare_result = pd.DataFrame({'house_id': validation_data_cleaned['house_id'], 'price': predictions})
    return compare_result

### Conduct the predictions

In [17]:
base_predictions = model_1()
compare_predictions = model_2()

# Print or inspect the predictions
print("Baseline Model Predictions:")
print(base_predictions.head())

print("\nComparison Model Predictions:")
print(compare_predictions.head())

Baseline Model Predictions:
      house_id          price
1116   1320704   42478.659727
1368   1089831  363069.027156
422    1639742  189047.285246
413    1421799  199220.224783
451    1378910   45595.711497

Comparison Model Predictions:
   house_id     price
0   1320704   35770.0
1   1089831  386001.0
2   1639742  209767.0
3   1421799  250696.0
4   1378910   53965.0
